# Why Fine-tuning
- To fit the requirement in specific professional fields
- The limitations from prompt engineering: requires too much input, output is not ideal
- To deploy locally for data safety
- Customized service

# Target of Fine-tuning
- To eliminate illusions from the model, improve response to instructions, work in specific professional fields
- Protect data safety and privacy
- Reduce training cose and improve training efficiency. Dynamicly learning.
- Improve reliability and response time

# Two Fine-tuning models
#### Incremental Pretraining
- Teach the base model a new knowledge，for specific professional fields
- Training object: article, book, code

#### Instruction Tuning
- Enable the model to learn the chat model, to communicate with humans per instructions
- Training object: q&a, high quality communications

# Lifecycle of a piece of data
- Raw data: involves system settinngs, user input and robot output
- Formalize data: convert the raw data in format of json
- Add dialog template: to identify if the words are from system, user or robot. Different models apply different templates.
- Tokenize data
- Add label
- Start training: only calculating loss by comparing robot output and expected output

# Fine-tuning Strategy
### Fine-tuning
- It involves base model in the training. It is necessary to save the optimizer state for the parameters in the base model.

### LoRA (Low rank adaptions of large language model)
- Add a new branch next to the original linear layer, consisting of two consecutive smaller linear layers. This new branch is called adapter. Adapter parameters are much smaller than the original linear layer, significantly reducing memory consumption during training
- Base model is only involved in forward training. Only the Adapter parameters are updated in backward training. Only optimizer state for parameters from Adapters are required to be saved.

### QLoRA
- Improvement based on LoRA
- Quantize the model size to 4-bit: reduce the precision of the model's weights and activations to 4 bits, which can significantly reduce the model's size and computational requirements, often with minimal impact on performance.

# Steps to take for fine tuning
- Clarify target application and task
- Select a base model
- Prepare data: collect, clean, pre-process, label, slice
    - Data collection: Online, public datasets(Alpaca, Self-instruct, LIMA, Dolly...), manual label
    - Data enforcement: rephrase data inputs, convert context to dialogs
    - Convert data format: via XTuner, divide training and test sets

###### Subsequent steps are supported by XTuner
- Set fine tuning strategy: LoRA, QLOra
- Set parameters: batch size, evaluation_inputs, evaluation_freq...
- Initialize the model
- Start fine tuning
###### -----
- Evaluate model and adjust parameters
- Test model performance
- Deploy

# InternLM Model - XTuner
- Compatible with various fine-tuning algorithms, covering a wide range of SFT scenarios
- Support HuggingFace Hub (utility based models), ModelScope
- Equip with map functions to transform format of datasets and dialog templates
- Require 8GB GPU memory

### Deploy
- Install xtuner and pick appropriate configuration template: xtuner list-cfg -p [model_base]
- Copy configuration template: xtuner copy-cfg [model_base]\_[algo]\_[data_sets]\_[data length]\_[Epoch] /dest/path
- Edit configuration template: data path or warehouse name, max length(maximum token number), pack to max length（boolean, whether to concat multi tokens to save, accumulative counts, evaluation inputs, evaluation freq
- Start training: xtuner train [model_base]\_[algo]\_[data_sets]\_[data length]\_[Epoch]\_copy.py

### Chat
- float 16: xtuner chat path/model_name
- 4bit: xtuner chat path/model_name --bits 4
- load adapter model: xtuner chat path/model_name --adapter [adapter_dir]

### Optimization Strategy to deploy Xtuner with 8GB GPU memory
#### Flash Attention (auto triggered)
- Parallel Attention Calculation (what is attention calculation? what's it for??), to reduce the N\*N VRAM occupization when calculating Attention Score

#### DeepSpeed ZeRO (triggered with \-\-deepspeed deepspeed_zero3)
- Slice and save parameters, gradients, and optimizer states during training to save VRAM when training on multiple GPUs
- Use FP16 weights with DeepSpeed training significantly saves VRAM on a single GPU compared to PyTorch's AMP training

# XTuner Project with QLoRA

- Mirror: Cuda 11.7 Conda
- GPU: 10% A100

### Steps: 
- https://github.com/InternLM/Tutorial/blob/camp2/xtuner/personal_assistant_document.md

#### Environment preparament
- Install xtuner and activate: studio-conda xtuner0.1.17, conda activate xtuner0.1.17
- Clone xtuner coda and install from cloned code
![alt text](../images/xtuner_llm_hw_pic1.png)

- Prepare training data in /root/ft/data/personal_assistant.json
- Prepare model: download from OpenXLab or Modelscope. In this project, it has been pre-installed, so just link from the root of virtual machine.

#### Configuration file setup
- list the supported confiduration template and copy the approriate one to /root/ft/config
![alt text](../images/xtuner_llm_hw_pic2.png)
- Edit Configuration template

#### Start training
- Can alternatively use deep-speed, if your local machine has a limited GPU source
![alt_text](../images/xtuner_llm_hw_pic3.png)
![alt_text](../images/xtuner_llm_hw_pic4.png)

#### Convertion, integration, test and deployment
- Convert the model weight files originally trained using PyTorch to the commonly used Huggingface format
- Integrate the trained adapter with the original model
![alt text](../images/xtuner_llm_hw_pic5.png)
- chat
![alt text](../images/xtuner_llm_hw_pic6.png)

#### Deploy to the web
- Install streamlit, clone InternLM code
- Use powershell ssh to the virtual machine
- Run web demo: streamlit run /root/ft/web_demo/InternLM/chat/web_demo.py --server.address 127.0.0.1 --server.port 6006

# InternLM2 1.8B Model
- InternLM2-Chat-1.8B-SFT: chat model by applying supervised fine-tuning (SFT) on InternLM2-1.8B
- InternLM2-Chat-1.8B (3.78GB): improve the model alignment and performance through Reinforcement Learning from Human Feedback (RLHF) techniques applied after initial supervised fine-tuning (SFT) on the InternLM2-Chat-1.8B model
- Under precision FP16, InternLM2-1.8B requires only 4GB VRAM to run on a pc, and 8GB VRAM to fine tune the model (beginner friendly) 

# Multimodal LLM
- For multiple types of data inputs, to perform tasks that require integrating information from these different modalities.

### LLaVA - enhance visual capability for LLM models
- Use GPT-4V generating image descriptions, in terms of "<question text\><image\> \-\- <answer text\>"
- Taking data sets in previous step, with text-only LLM, to train an Image Projector. The image Projector and text-only LLM are called LLaVA model

#### Deploy
- Pretrain: use images with title and texts and text-only LLM (i.e. InterLM2_chat_1.8B) to generate a pretrained LLaVA
- Finetune: use the pretrained LLaVA and images with complicated chats to generate Finetuned LLaVA